In [40]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import tweepy
import time
import pandas as pd
from selenium import webdriver


In [77]:
driver = webdriver.Chrome('C:/Users/victo/ChromeDriver/chromedriver.exe') 

#### Step one is to visit the NASA Mars News Site to scrape the title/and paragraph text. 

In [82]:
driver = webdriver.Chrome('C:/Users/victo/ChromeDriver/chromedriver.exe') 
mars_news = 'https://mars.nasa.gov/news/'
driver.get(mars_news)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
news_title = soup.find("div", class_ = "content_title").text
news_content = soup.find("div", class_ = "article_teaser_body").text
news = [news_title, news_content]
print(news)
driver.quit()

["NASA's InSight Passes Halfway to Mars, Instruments Check In", "NASA's InSight spacecraft, en route to a Nov. 26 landing on Mars, passed the halfway mark on Aug. 6. All of its instruments have been tested and are working well."]


#### Step 2 is to visit JPL site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [79]:
driver = webdriver.Chrome('C:/Users/victo/ChromeDriver/chromedriver.exe') 
mars_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
driver.get(mars_image)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
images = soup.find("a", class_ = "button fancybox")
featured_image_url = mars_image + images.get('data-fancybox-href')
print(featured_image_url)
driver.quit()

https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/mediumsize/PIA16021_ip.jpg


#### Next we gotta visit the official NASA Mars twitter account to scrape the latest weather tweet. 

In [67]:
driver = webdriver.Chrome('C:/Users/victo/ChromeDriver/chromedriver.exe') 
twitter_url = "https://twitter.com/marswxreport?lang=en"
html = driver.page_source
response = requests.get(twitter_url)
soup = BeautifulSoup(response.text,"lxml")
tweets = soup.findAll('li',{"class":'js-stream-item'})
tweet_records = []
for tweet in tweets:
        if tweet.find('p',{"class":'tweet-text'}):
            tweet_text = tweet.find('p',{"class":'tweet-text'}).text.encode('utf8').strip()
        tweet_records.append(tweet_text)
mars_weather_status = tweet_records[1]
print(mars_weather_status)
driver.quit()

b'Sol 2144 (2018-08-18), high -11C/12F, low -67C/-88F, pressure at 8.67 hPa, daylight 05:29-17:42'


#### We have almost all the data that we want, at this point we are going to scrape the table containing facts about the planet including Diameter, Mass, etc from the mars facts page, let's also go ahead and use pandas' to html method to put the scraped facts into an html string. 

In [68]:
driver = webdriver.Chrome('C:/Users/victo/ChromeDriver/chromedriver.exe') 
facts = requests.get("https://space-facts.com/mars/")
soup = BeautifulSoup(facts.content, 'lxml')
mars_table = soup.find_all('table')[0]
mars_data = pd.read_html(str(mars_table))[0]
mars_data.columns = ["Description", "Value"]
mars_data = mars_data.set_index("Description")
mars_facts = mars_data.to_html(index = True, header =True)
print(mars_facts)
driver.quit()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


#### Last thing we are going to do is to scrape some high resolution images for each of Mar's hemispheres from the USGS Astrogeology page. I am going to save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name and I am going use a Python dictionary to store the data using the keys img_url and title. Lastly, I am going to append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere

In [85]:
driver = webdriver.Chrome('C:/Users/victo/ChromeDriver/chromedriver.exe') 
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver.get(hemisphere_url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

mars_hemisphere_list = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

for hemisphere in hemispheres:
        title = hemisphere.find("h3").text
        title = title.replace("Enhanced", "")
        end_link = hemisphere.find("a")["href"]
        image_url = "https://astrogeology.usgs.gov/" + end_link
        mars_hemisphere_list.append({"title": title, "img_url": image_url})
def get_high_res_url(some_url):
        response = requests.get(some_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all("a")
        tifs = [j for j in links if ".tif" in j.attrs.get('href')]
        return tifs[0].get('href')

updated_photos = []

for data in mars_hemisphere_list:
        link_to_check = data.get('img_url')
        title = data.get('title')
        final_image_url = get_high_res_url(link_to_check)
        updated_photos.append({
            'Title': title,
            'Url': final_image_url
        })

print(final_image_url)
driver.quit()

http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
